In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet50
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from torchvision.models import vgg16

import torch.optim as optim
import torch.nn as nn
import time
import os
from torch.utils.data import Subset
import random
import argparse

def get_random_subset_indices(num_samples, dataset_size):
    return random.sample(range(dataset_size), num_samples)


def create_dataset():
     # Data augmentation and normalization for training
    train_transform = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    

    train_dataset = torchvision.datasets.ImageFolder(os.path.join("/lus/eagle/projects/datascience/ImageNet/ILSVRC/Data/CLS-LOC", "train"), transform=train_transform)

    num_samples = 10000

    train_indices = get_random_subset_indices(num_samples, len(train_dataset))
    small_train_dataset = Subset(train_dataset, train_indices)
    return small_train_dataset

def build_model():
    model = vgg16(weights=None)
    num_ftrs = model.classifier[6].in_features
    model.classifier[6] = nn.Linear(num_ftrs, 1000)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=1e-4)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model = nn.DataParallel(model)
    return model, criterion, optimizer, device


def create_dataLoader(batch_size=128, workers=4):
    return DataLoader(create_dataset(), batch_size=batch_size, num_workers=workers, shuffle=True, pin_memory=True)

def select_device(selected_gpus):
     os.environ["CUDA_VISIBLE_DEVICES"] = ",".join(map(str, selected_gpus))

def train_one_epoch(model, criterion, optimizer, data_loader, device):
    model.train()

    total_images = 0
    start_time = time.time()
    start_time_dataLoad = time.time()
    end_time_dataLoad = 0
    for i, (inputs, labels) in enumerate(data_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        total_images += inputs.size(0)

    end_time = time.time()
    images_per_second = total_images / (end_time - start_time)
    dataLoad_time = end_time_dataLoad - start_time_dataLoad
    return int(images_per_second), end_time-start_time


def train(batch_size=256, GPU_selection=[0, 1], epoch=5, num_workers=8):
    
    num_epochs = epoch  # Adjust this value according to your needs
    train_loader = create_dataLoader(batch_size, num_workers)
    select_device(GPU_selection)
    model,criterion,optimizer,device = build_model()
    
    
    for epoch in range(num_epochs):
        images_per_second, epoch_duration = train_one_epoch(model, criterion, optimizer, train_loader, device)
      
        print(f"Epoch [{epoch + 1}/{num_epochs}],Duration: {epoch_duration:.2f}s, Images/s: {images_per_second}")

In [ ]:
train(batch_size=64, GPU_selection=[0], epoch=5, num_workers=8)

In [ ]:
train(batch_size=64, GPU_selection=[0,1], epoch=5, num_workers=8)

In [ ]:
train(batch_size=64, GPU_selection=[0,1,2], epoch=5, num_workers=8)

In [2]:
train(batch_size=256, GPU_selection=[0,1,2,3], epoch=5, num_workers=4)

Epoch [1/5],Duration: 72.02s, Images/s: 138
Epoch [2/5],Duration: 9.36s, Images/s: 1068
Epoch [3/5],Duration: 9.23s, Images/s: 1083
Epoch [4/5],Duration: 9.71s, Images/s: 1029
Epoch [5/5],Duration: 9.14s, Images/s: 1094
